На основе рассмотренного на занятии Colab-ноутбука по распознаванию предметов одежды решите задачу распознавания рукописных цифр. 

На этот раз примените не полносвязную, а сверточную нейронную сеть. 

Сравните полученные результаты работы сверточной нейронной сети с аналогичными результатами работы полносвязной нейронной сети.

Поэкспериментируйте с различными конфигурациями сети. 

Приведите графики долей правильных ответов на обучающем множестве и валидационном. 

Определите оптимальное количество эпох для обучения для каждой из используемых конфигураций нейронной сети. 

Получите оценки качества классификаторов на тестовом множестве. 

Какая конфигурация из используемых лучше? 

Определите для каждой цифры на тестовом множестве точность работы классификаторов. 

Проведите тестирование построенной модели на реальных данных с изображениями рукописных цифр.

In [2]:
from tensorflow import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras import utils
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [3]:
classes = ['class_0', 'class_1', 'class_2', 'class_3', 'class_4', 'class_5', 'class_6', 'class_7', 'class_8', 'class_9']

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [5]:
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))
x_train, x_test = x_train / 255.0, x_test / 255.0

In [6]:
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [7]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),#первый слой
    #сделали изображение ЧБ
    #relu - функция активации
    #у слоя 32 фильтра, у такого же следующего 64
    #размеры фильров 3 на 3
    MaxPooling2D((2,2)),#слой обьеденения макс значения
    Conv2D(64, (3,3), activation='relu'),#ещё свёрточный слой
    MaxPooling2D((2,2)),#ещё обьеденение
    Flatten(),#преобразование в вектор
    Dense(64, activation='relu'),#слой с 64 нейронами
    Dense(10, activation='softmax')#выходной слой с 10 нейронами
])

In [8]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Epoch 1/5
1875/1875 [==============================] - 20s 10ms/step - loss: 0.1414 - accuracy: 0.9573 - val_loss: 0.0731 - val_accuracy: 0.9768
Epoch 2/5
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0470 - accuracy: 0.9857 - val_loss: 0.0454 - val_accuracy: 0.9851
Epoch 3/5
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0338 - accuracy: 0.9894 - val_loss: 0.0379 - val_accuracy: 0.9883
Epoch 4/5
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0253 - accuracy: 0.9922 - val_loss: 0.0357 - val_accuracy: 0.9890
Epoch 5/5
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0193 - accuracy: 0.9937 - val_loss: 0.0315 - val_accuracy: 0.9904


по результатм обучения видно, что хватает и 2 эпох

In [10]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('точность:', test_acc)

313/313 [==============================] - 1s 3ms/step - loss: 0.0315 - accuracy: 0.9904
точность: 0.9904000163078308


In [11]:
def preobr(img):
    x = utils.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x /= 255
    return x

In [12]:
for i in range(10):
    img = utils.load_img(f"img\\{i}.jpg", target_size=(28, 28), color_mode = "grayscale")
    prediction = model.predict(preobr(img))
    #print(prediction)
    pred = np.argmax(prediction)
    con = prediction[0][pred]
    print(f"взяли : {i}")
    print("Название класса:", classes[pred])
    print(f"уверенность : {con}")

1/1 [==============================] - 0s 93ms/step
взяли : 0
Название класса: class_0
уверенность : 0.96840500831604
1/1 [==============================] - 0s 19ms/step
взяли : 1
Название класса: class_6
уверенность : 0.5610037446022034
1/1 [==============================] - 0s 20ms/step
взяли : 2
Название класса: class_4
уверенность : 0.919871985912323
1/1 [==============================] - 0s 20ms/step
взяли : 3
Название класса: class_4
уверенность : 0.6366264820098877
1/1 [==============================] - 0s 20ms/step
взяли : 4
Название класса: class_4
уверенность : 0.9927324652671814
1/1 [==============================] - 0s 20ms/step
взяли : 5
Название класса: class_5
уверенность : 0.44892072677612305
1/1 [==============================] - 0s 20ms/step
взяли : 6
Название класса: class_5
уверенность : 0.9489426016807556
1/1 [==============================] - 0s 20ms/step
взяли : 7
Название класса: class_4
уверенность : 0.7619015574455261
1/1 [==============================] - 0s 

видим, что на многих цифрах плохие резьтаты, возможно из-за переобучения

In [13]:
model.fit(x_train, y_train, epochs=1, validation_data=(x_test, y_test))

1875/1875 [==============================] - 19s 10ms/step - loss: 0.0152 - accuracy: 0.9949 - val_loss: 0.0241 - val_accuracy: 0.9923


In [14]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('точность:', test_acc)

313/313 [==============================] - 1s 3ms/step - loss: 0.0241 - accuracy: 0.9923
точность: 0.9922999739646912


In [15]:
for i in range(10):
    img = utils.load_img(f"img\\{i}.jpg", target_size=(28, 28), color_mode = "grayscale")
    prediction = model.predict(preobr(img))
    #print(prediction)
    pred = np.argmax(prediction)
    con = prediction[0][pred]
    print(f"взяли : {i}")
    print("Название класса:", classes[pred])
    print(f"уверенность : {con}")

1/1 [==============================] - 0s 20ms/step
взяли : 0
Название класса: class_0
уверенность : 0.9122517108917236
1/1 [==============================] - 0s 20ms/step
взяли : 1
Название класса: class_6
уверенность : 0.7863438725471497
1/1 [==============================] - 0s 20ms/step
взяли : 2
Название класса: class_4
уверенность : 0.5647439360618591
1/1 [==============================] - 0s 20ms/step
взяли : 3
Название класса: class_8
уверенность : 0.37968331575393677
1/1 [==============================] - 0s 20ms/step
взяли : 4
Название класса: class_4
уверенность : 0.9888566136360168
1/1 [==============================] - 0s 19ms/step
взяли : 5
Название класса: class_4
уверенность : 0.3656160533428192
1/1 [==============================] - 0s 19ms/step
взяли : 6
Название класса: class_5
уверенность : 0.7653658390045166
1/1 [==============================] - 0s 20ms/step
взяли : 7
Название класса: class_7
уверенность : 0.3934202790260315
1/1 [==============================] - 

и всёравно не особо хорошие результаты ((

Пробую пересоздать модель с использованием Dropout
это позволит в случайном порядке отключать 20% нейронов для избежания переобучения 

так же использую уже готовый обученный слой для распознования картинок, правда пришлось поменять размер картинок 32 на 32

In [16]:
from keras.layers import Dropout, GlobalAveragePooling2D

In [17]:
base_layers = keras.applications.MobileNetV2(input_shape=(32, 32, 3), include_top=False) # заготовленная модель, где обученны перввые слои
base_layers.trainable = False #говорим не переобучать первые слои

In [18]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')
])

In [19]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

Epoch 1/2
1875/1875 [==============================] - 20s 11ms/step - loss: 0.1719 - accuracy: 0.9480 - val_loss: 0.0506 - val_accuracy: 0.9838
Epoch 2/2
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0625 - accuracy: 0.9814 - val_loss: 0.0336 - val_accuracy: 0.9883


In [21]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('точность:', test_acc)

313/313 [==============================] - 1s 4ms/step - loss: 0.0336 - accuracy: 0.9883
точность: 0.9883000254631042


In [22]:
for i in range(10):
    img = utils.load_img(f"img\\{i}.jpg", target_size=(28, 28), color_mode = "grayscale")
    prediction = model.predict(preobr(img))
    #print(prediction)
    pred = np.argmax(prediction)
    con = prediction[0][pred]
    print(f"взяли : {i}")
    print("Название класса:", classes[pred])
    print(f"уверенность : {con}")

1/1 [==============================] - 0s 68ms/step
взяли : 0
Название класса: class_0
уверенность : 0.592850923538208
1/1 [==============================] - 0s 19ms/step
взяли : 1
Название класса: class_4
уверенность : 0.4806431829929352
1/1 [==============================] - 0s 21ms/step
взяли : 2
Название класса: class_4
уверенность : 0.8199167847633362
1/1 [==============================] - 0s 20ms/step
взяли : 3
Название класса: class_9
уверенность : 0.3817760646343231
1/1 [==============================] - 0s 19ms/step
взяли : 4
Название класса: class_4
уверенность : 0.7901642918586731
1/1 [==============================] - 0s 20ms/step
взяли : 5
Название класса: class_0
уверенность : 0.6601783633232117
1/1 [==============================] - 0s 21ms/step
взяли : 6
Название класса: class_8
уверенность : 0.31316816806793213
1/1 [==============================] - 0s 20ms/step
взяли : 7
Название класса: class_4
уверенность : 0.5507037043571472
1/1 [==============================] - 0

всё стало только хуже(((